In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

from IPython import get_ipython
import argparse
import numpy as np
import pandas as pd
from pandas import DataFrame
import time
import torch
import matplotlib as mpl

## Process the drugs' SMILE string 

In [3]:
drug_resp = pd.read_csv('./data/primary-screen-replicate-collapsed-logfold-change.csv', index_col=0)
# drug_resp = pd.read_csv('./data/sanger-dose-response.csv')
drug_resp.index = [s.split('_')[0] for s in drug_resp.index]

# rows: cell lines, columns: drugs
drug_resp.head()

,BRD-A00077618-236-07-6::2.5::HTS,BRD-A00100033-001-08-9::2.5::HTS,BRD-A00147595-001-01-5::2.5::HTS,BRD-A00218260-001-03-4::2.5::HTS,BRD-A00376169-001-01-6::2.5::HTS,BRD-A00520476-001-07-4::2.5::HTS,BRD-A00546892-001-02-6::2.5::HTS,BRD-A00578795-001-04-3::2.5::HTS,BRD-A00758722-001-04-9::2.5::HTS,BRD-A00827783-001-24-6::2.5::HTS,...,BRD-K98557884-001-01-6::2.5::MTS004,BRD-K99077012-001-01-9::2.332734192::MTS004,BRD-K99199077-001-16-1::2.603211317::MTS004,BRD-K99431849-001-01-7::2.500018158::MTS004,BRD-K99447003-335-04-1::2.37737659::MTS004,BRD-K99506538-001-03-8::2.5::MTS004,BRD-K99616396-001-05-1::2.499991421::MTS004,BRD-K99879819-001-02-1::2.5187366::MTS004,BRD-K99919177-001-01-3::2.5::MTS004,BRD-M63173034-001-03-6::2.64076472::MTS004
ACH-000001,-0.015577,-0.449332,0.489379,0.206675,0.272730,0.021036,-0.025460,0.467158,-0.736306,0.644137,...,0.429238,0.204841,0.150055,-0.575404,-0.101247,0.399233,-0.127658,-0.141651,-1.153652,0.510464
ACH-000007,-0.095730,0.257943,0.772349,-0.438502,-0.732832,0.779201,0.426523,-1.288508,-0.476133,-0.277105,...,-0.471486,0.212998,-0.123230,0.625527,0.383198,0.212031,0.349225,-0.387439,-0.831461,0.323558
ACH-000008,0.379480,-0.596132,0.548056,0.422269,-0.216986,0.081866,0.145335,-0.570841,-0.512119,0.452698,...,-0.111951,0.534787,0.206642,-0.410153,-0.560722,-0.036088,0.158071,0.171043,-3.947090,0.099310
ACH-000010,0.118890,-0.231615,0.621937,-0.202707,-1.005139,-0.213739,0.020246,-0.795278,NaN,0.679571,...,0.200605,-0.075356,0.610310,-0.019413,-0.202971,0.218158,-0.411009,-0.181540,-3.010225,0.090652
ACH-000011,0.145346,-0.499274,0.267470,0.157804,-0.272286,0.207768,0.004464,-0.191680,-0.310375,0.112537,...,-0.076863,0.026002,0.139921,-0.261704,0.085339,0.447482,0.164620,-0.565251,-4.110627,0.222394


In [4]:
drg_brd_id = drug_resp.columns.values
drg_brd_id = [s for s in drg_brd_id if "BRD-" in s]
drg_brd_id = [s.split(':')[0] for s in drg_brd_id]
len(drg_brd_id)

4686

In [5]:
drugs_info = pd.read_csv('./data/primary-screen-replicate-treatment-info.csv', sep=',')
drugs_info = drugs_info[~drugs_info.smiles.isna() ]
drugs_info.head()

,column_name,broad_id,name,dose,perturbation_type,Unnamed: 5,Unnamed: 6,screen_id,detection_plate,compound_plate,well,detection_plate_scan_time,vendor_id,purity,moa,target,disease.area,indication,smiles,phase
2,PREP001_X1:A03::HTS,BRD-A96485169-001-14-3,EBPC,2.5,experimental_treatment,NaN,NaN,HTS,PREP001_X1,PREP001,A03,2/11/16 10:09,518,91.11,aldose reductase inhibitor,AKR1B1,NaN,NaN,CCOC(=O)C1CN(Cc2ccccc2)C(=O)C1=O,Preclinical
3,PREP001_X1:A04::HTS,BRD-K35559145-050-08-5,levomepromazine,2.5,experimental_treatment,NaN,NaN,HTS,PREP001_X1,PREP001,A04,2/11/16 10:09,Prestw-797,76.90,dopamine receptor antagonist,"ADRA1A, ADRA1B, ADRA1D, ADRA2A, ADRA2B, ADRA2C...","neurology/psychiatry, gastroenterology","psychosis, schizophrenia, bipolar disorder, na...","COc1ccc2Sc3ccccc3N(C[C@H](C)CN(C)C)c2c1, COc1c...",Launched
4,PREP001_X1:A05::HTS,BRD-A48015106-004-08-9,8-hydroxy-DPAT,2.5,experimental_treatment,NaN,NaN,HTS,PREP001_X1,PREP001,A05,2/11/16 10:09,529,93.13,serotonin receptor agonist,"CDK1, CHEK1, CHEK2, GSK3B, HTR5A, HTR7, LCK, M...",NaN,NaN,CCCN(CCC)C1CCc2cccc(O)c2C1,Preclinical
5,PREP001_X1:A06::HTS,BRD-A59198242-003-02-9,viloxazine,2.5,experimental_treatment,NaN,NaN,HTS,PREP001_X1,PREP001,A06,2/11/16 10:09,Prestw-1361,98.75,norepinephrine reuptake inhibitor,SLC6A2,neurology/psychiatry,depression,CCOc1ccccc1OCC1CNCCO1,Launched
6,PREP001_X1:A07::HTS,BRD-A30590053-003-02-9,MR-16728,2.5,experimental_treatment,NaN,NaN,HTS,PREP001_X1,PREP001,A07,2/11/16 10:09,537,93.83,acetylcholine release enhancer,NaN,NaN,NaN,O=C(NCCCN1CCCCCC1)C(C1CCCCC1)c1ccccc1,Preclinical


In [6]:
drugs_info.shape

(14700, 20)

In [7]:
drugs_info_broad_id = drugs_info[~drugs_info.broad_id.isna()]['broad_id'].values

In [38]:
# Number of drugs in the study with SMILE string
len(set(drg_brd_id).intersection(set(drugs_info_broad_id)))

4684

In [15]:
# Only two drugs in the study without the SMILE string
set(drg_brd_id) - set(drg_brd_id).intersection(set(drugs_info_broad_id))

{'BRD-K28042756-001-01-9', 'BRD-U08520523-000-01-0'}

In [39]:
drugs_info_final = drugs_info[drugs_info.broad_id.isin(drg_brd_id)].drop_duplicates(subset='broad_id', keep='first')
drugs_info_final['SMILE'] = drugs_info_final['smiles'].apply(lambda s: s.split(',')[0].strip() )

In [41]:
drugs_info_final.to_csv('./data/screen_smiles.csv', columns=['broad_id', 'smiles', 'SMILE'], index=False)

## Process cell lines gene expression

In [39]:
data = pd.read_csv('./data/CCLE_expression.csv', index_col=0)

In [6]:
data.head()

,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),NFYA (4800),...,ARHGAP11B (89839),AC004593.2 (1124),AC090517.4 (54816),AL160269.1 (11046),ABCF2-H2BE1 (114483834),POLR2J3 (548644),H2BE1 (114483833),AL445238.1 (647264),GET1-SH3BGR (106865373),AC113348.1 (102724657)
ACH-000784,3.745237,0.000000,6.767655,2.292782,2.395063,0.028569,0.056584,0.176323,9.289420,3.643856,...,1.104337,0.070389,0.056584,0.201634,2.735522,4.720826,0.333424,0.0,0.214125,0.000000
ACH-000249,3.896272,0.000000,6.063071,1.941106,3.350497,0.056584,0.422233,5.526382,3.955127,3.693766,...,1.367371,0.111031,0.042644,0.042644,4.097611,6.507636,0.000000,0.0,1.028569,0.000000
ACH-000974,5.252855,0.000000,5.974529,1.726831,2.333424,0.000000,0.097611,5.513807,3.423578,4.312520,...,1.195348,0.176323,0.070389,0.042644,0.000000,5.305241,0.111031,0.0,0.739848,0.000000
ACH-000443,4.853497,0.028569,6.474598,3.001802,4.100137,0.895303,4.165912,4.721373,3.450221,3.757023,...,2.922198,0.084064,0.042644,0.963474,2.704872,5.831877,0.000000,0.0,1.104337,0.000000
ACH-000955,5.356848,0.286881,5.860466,4.056584,4.785027,0.042644,1.718088,5.004052,5.780835,5.096346,...,2.392317,0.124328,0.070389,0.000000,2.192194,6.111657,0.000000,0.0,0.669027,0.042644


In [40]:
# Load CGC genes
cgc = pd.read_csv('./data/Census_allThu Jul 30 07_07_22 2020.csv')

In [41]:
genes = [s.split(' ')[0] for s in data.columns.values]

# Number of CGC genes in the cell line genes set: 704/723 
len(set(genes).intersection(set(cgc['Gene Symbol'].values))), len(set(cgc['Gene Symbol'].values))

(704, 723)

In [42]:
filter_genes_indx = [i for i in range(len(genes)) if genes[i] in cgc['Gene Symbol'].values]

In [25]:
# save the index of the columns for the 704 CGC genes
with open('./data/GCG_filter_indx.txt', 'w') as f:
    f.write('\n'.join([str(s) for s in filter_genes_indx]))

In [14]:
# CGC genes not in the cell lines gene set
set(cgc['Gene Symbol'].values) - set(genes).intersection(set(cgc['Gene Symbol'].values))

{'CARS',
 'DUX4L1',
 'FGFR1OP',
 'H3F3A',
 'H3F3B',
 'HIST1H3B',
 'HIST1H4I',
 'HMGN2P46',
 'IGH',
 'IGK',
 'IGL',
 'MALAT1',
 'MDS2',
 'SEPT5',
 'SEPT6',
 'SEPT9',
 'TRA',
 'TRB',
 'TRD'}

In [43]:
# Save the gene expression profiles for the cell lines in our study
keys = list(set(data.index.values).intersection(set(drug_resp.index.values)))
data.loc[keys,].to_csv('./data/CCLE_expression_full_final.csv')

## Process Mutation Data

In [15]:
ccle_mutation = pd.read_csv('./data/CCLE_mutations.csv')

/home/labuser/anaconda3/envs/rdkit/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (3,19,22,27,28,29,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
ccle_mutation.head()

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,isCOSMIChotspot,COSMIChsCnt,ExAC_AF,Variant_annotation,CGA_WES_AC,HC_AC,RD_AC,RNAseq_AC,SangerWES_AC,WGS_AC
0,VPS13D,55187,37,1,12359347,12359347,+,Nonsense_Mutation,SNP,C,...,False,0.0,NaN,damaging,34:213,NaN,NaN,NaN,34:221,NaN
1,AADACL4,343066,37,1,12726308,12726322,+,In_Frame_Del,DEL,CTGGCGTGACGCCAT,...,False,3.0,NaN,other non-conserving,57:141,NaN,NaN,NaN,9:0,28:32
2,IFNLR1,163702,37,1,24484172,24484172,+,Silent,SNP,G,...,False,0.0,NaN,silent,118:0,NaN,NaN,10:0,118:0,18:0
3,TMEM57,55219,37,1,25785018,25785019,+,Frame_Shift_Ins,INS,-,...,False,0.0,NaN,damaging,NaN,NaN,NaN,6:28,NaN,NaN
4,ZSCAN20,7579,37,1,33954141,33954141,+,Missense_Mutation,SNP,T,...,False,0.0,NaN,other non-conserving,28:62,NaN,NaN,NaN,27:61,NaN


In [28]:
df = pd.crosstab(ccle_mutation.DepMap_ID, ccle_mutation.Hugo_Symbol)
df.head()

Hugo_Symbol,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000002,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
ACH-000003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
ACH-000004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# Save just the mutation data for the cell lines in the sensitivity-drug study
keys = list(set(data.index.values).intersection(set(df.index.values)))
df.loc[keys, ].to_csv('./data/CCLE_muttion_final.csv')